In [1]:
%matplotlib inline

import calendar
import datetime
import re

import nltk
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup
from imblearn.over_sampling import SMOTE
from skopt import BayesSearchCV
import seaborn as sns; sns.set(color_codes=True)
import matplotlib.pyplot as plt

from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from scipy import stats
from scipy.stats import randint, uniform

from sklearn import linear_model, preprocessing, svm
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import Lasso, LogisticRegression, Ridge
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, mean_absolute_error,
                             mean_squared_error, r2_score)
from sklearn.model_selection import (GridSearchCV, RandomizedSearchCV,
                                     train_test_split)
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import (MinMaxScaler, PolynomialFeatures,
                                   RobustScaler, StandardScaler)
from sklearn.svm import SVC

Using TensorFlow backend.


In [2]:
dataset = pd.read_csv('vision_sample.csv')
dataset.head()

,External ID,Naam,Postcode,Plaats,Geslacht,Geboortedatum,Oogmetingen/Datum,Oogmetingen/Sfr ver,Oogmetingen/Cyl ver,Oogmetingen/As ver,...,Oogmetingen/Sfr dicht,Oogmetingen/Cyl dicht,Oogmetingen/As dicht,Oogmetingen/Sfr ver.1,Oogmetingen/Cyl ver.1,Oogmetingen/As ver.1,Oogmetingen/Add.1,Oogmetingen/Sfr dicht.1,Oogmetingen/Cyl dicht.1,Oogmetingen/As dicht.1
0,__export__.res_partner_800_7573293f,Anthony Richardson,NaN,Willemstad,Man,3/10/1980,3/31/2020,1.25,0.00,0,...,1.75,0.00,0,0.75,0.00,0,0.50,1.25,0.00,0
1,NaN,NaN,NaN,NaN,NaN,NaN,1/19/2018,1.25,0.00,0,...,1.75,0.00,0,0.75,0.00,0,0.50,1.25,0.00,0
2,__export__.res_partner_796_e52c86ad,Peters Dominique,9031.0,Drongen,Man,5/19/1980,3/23/2020,1.75,1.25,120,...,4.25,1.25,120,0.00,0.00,0,2.50,2.50,0.00,0
3,NaN,NaN,NaN,NaN,NaN,NaN,3/30/2016,2.50,1.25,120,...,4.75,1.25,120,0.00,0.00,0,2.25,2.25,0.00,0
4,NaN,NaN,NaN,NaN,NaN,NaN,4/1/2012,1.00,1.25,80,...,1.50,1.25,80,2.00,2.25,180,0.50,2.50,2.25,180


In [6]:
for i in range(len(dataset.columns.values) - 1):
    dataset.columns.values[i-1] = dataset.columns.values[i-1].replace('Oogmetingen/', '')

dataset.drop(['External ID', 'Naam', 'Postcode', 'Plaats'], axis=1, inplace=True)
dataset.head()

NameError: name 'dataset' is not defined